In [ ]:
import numpy as np
import sys
sys.path.append(r"C:\Users\gapar\Logical-Clifford-Synthesis")
from Algorithms.algorithm_1 import SymplecticOperations
from Algorithms.algorithm_2 import FindAllSympMat
from Algorithms.algorithm_3 import SymplecticCodeSolver
from helper_functions.helperfunctions import SymplecticMatrix
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# Initialize the SymplecticOperations class
symplectic_ops = SymplecticOperations()
symp_mat = SymplecticMatrix(4)
symp_code = SymplecticCodeSolver()
solver = FindAllSympMat()

In [ ]:
import numpy as np

def calc_conjugate(m, op_in, circuit):
    """
    Compute the effect of a Clifford circuit on an input Pauli operator under conjugation.
    Parameters:
    - m: Number of qubits.
    - op_in: Input Pauli operator as a tuple ('XYZ...', [qubit_indices]).
    - circuit: List of gates applied as tuples (Gate, [qubit_indices]).
    
    Returns:
    - op_out: Output Pauli operator in the same format as op_in.
    """
    # Define Pauli matrices and other gates
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    Y = 1j * X @ Z
    P = np.array([[1, 0], [0, 1j]], dtype=complex)
    H = (X + Z) / np.sqrt(2)

    e0 = np.array([[1], [0]], dtype=complex)
    e1 = np.array([[0], [1]], dtype=complex)
    E00 = e0 @ e0.T
    E11 = e1 @ e1.T
    CNOT = np.kron(E00, I) + np.kron(E11, X)
    CZ = np.kron(E00, I) + np.kron(E11, Z)

    # Initialize the input Pauli operators
    Pauli_in = [I] * m
    for i in range(m):
        if i + 1 in op_in[1]:
            Pli = op_in[0][op_in[1].index(i + 1)]
            Pauli_in[i] = {'X': X, 'Y': Y, 'Z': Z}.get(Pli, I)

    Pauli_out = Pauli_in[:]

    # Apply each gate in the circuit
    for gate, qubits in circuit:
        qubits = [q - 1 for q in qubits]  # Adjust to 0-indexing

        if gate == 'X':
            for q in qubits:
                Pauli_out[q] = X @ Pauli_out[q] @ X

        elif gate == 'Z':
            for q in qubits:
                Pauli_out[q] = Z @ Pauli_out[q] @ Z

        elif gate == 'Y':
            for q in qubits:
                Pauli_out[q] = Y @ Pauli_out[q] @ Y

        elif gate == 'P':
            for q in qubits:
                Pauli_out[q] = P @ Pauli_out[q] @ P

        elif gate == 'H':
            for q in qubits:
                Pauli_out[q] = H @ Pauli_out[q] @ H

        elif gate == 'CNOT':
            if len(qubits) != 2:
                raise ValueError("CNOT gate requires exactly two qubits.")
            control, target = qubits
            apply_cnot(Pauli_out, control, target, CNOT)

        elif gate == 'CZ':
            if len(qubits) != 2:
                raise ValueError("CZ gate requires exactly two qubits.")
            control, target = qubits
            apply_cz(Pauli_out, control, target, CZ)

        elif gate == 'Permute':
            desired_order = [q - 1 for q in qubits]
            if len(desired_order) != m:
                raise ValueError(f"Permutation requires {m} qubits.")
            Pauli_out = [Pauli_out[q] for q in desired_order]

    # Compute the final Pauli operator and sign
    op_out, out_sign = extract_pauli_operator(Pauli_out)

    # Adjust the sign
    if out_sign == -1:
        op_out = ('-' + op_out[0], [-1] + op_out[1])
    elif out_sign == 1j:
        op_out = ('i' + op_out[0], [1j] + op_out[1])
    elif out_sign == -1j:
        op_out = ('j' + op_out[0], [-1j] + op_out[1])

    return op_out

def apply_cnot(Pauli_out, control, target, CNOT):
    """Apply the CNOT gate."""
    out12 = find_other(CNOT, Pauli_out[control], 1)
    out21 = find_other(CNOT, Pauli_out[target], 2)
    Pauli_out[control] = out21 @ Pauli_out[control]
    Pauli_out[target] = Pauli_out[target] @ out12

def apply_cz(Pauli_out, control, target, CZ):
    """Apply the CZ gate."""
    out12 = find_other(CZ, Pauli_out[control], 1)
    out21 = find_other(CZ, Pauli_out[target], 2)
    Pauli_out[control] = out21 @ Pauli_out[control]
    Pauli_out[target] = Pauli_out[target] @ out12

def find_other(gate, inp, id):
    """Find the other output for a 2-qubit gate."""
    I = np.eye(2)
    Pin = np.kron(inp, I) if id == 1 else np.kron(I, inp)
    candidates = [np.kron(X, I), np.kron(Z, I), np.kron(Y, I), Pin]

    for candidate in candidates:
        if np.allclose(gate @ Pin @ gate.T.conj(), candidate):
            return candidate

    raise ValueError("Unknown gate encountered in find_other.")

def extract_pauli_operator(Pauli_out):
    """Extract the final Pauli operator and its sign."""
    op_out = ('', [])
    out_sign = 1

    for i, P in enumerate(Pauli_out):
        if np.allclose(P, X):
            op_out = (op_out[0] + 'X', op_out[1] + [i + 1])
        elif np.allclose(P, -X):
            op_out = (op_out[0] + 'X', op_out[1] + [i + 1])
            out_sign *= -1
        elif np.allclose(P, 1j * X):
            op_out = (op_out[0] + 'X', op_out[1] + [i + 1])
            out_sign *= 1j
        elif np.allclose(P, Z):
            op_out = (op_out[0] + 'Z', op_out[1] + [i + 1])
        elif np.allclose(P, Y):
            op_out = (op_out[0] + 'Y', op_out[1] + [i + 1])
        elif np.allclose(P, -Y):
            op_out = (op_out[0] + 'Y', op_out[1] + [i + 1])
            out_sign *= -1

    return op_out, out_sign

def mod2(matrix):
    """Perform element-wise mod 2 operation."""
    return np.mod(matrix, 2)

def find_symplectic(m, circuit):
    """Placeholder for the function that finds the symplectic matrix."""
    return SymplecticMatrix.find_symplectic(circuit)
def find_circuit(F):
    """Placeholder for the function to find the circuit for a symplectic matrix."""
    pass  # Implement this function based on your requirements

def gflineq(A, b):
    """Placeholder for solving linear equations in GF(2)."""
    return symplectic_ops.gf2_gaussian_elimination_with_echelon(A,b)

def qfind_all_symp_mat(U, H):
    """Placeholder for finding all symplectic matrices."""
    return symp_code.symplectic_code(U,H)

def get_conjugate_sign(j, m, circuit, pauli_type):
    """Calculate the sign under conjugation."""
    conj_ckt = calc_conjugate(m, [pauli_type, j], circuit)
    return -1 if conj_ckt[0][0] == '-' else 1

def convert_to_circuit(h, m):
    """Convert a complex vector to a circuit representation."""
    circuit = ['', []]
    for q in range(m):
        if h[q] == 1:
            circuit[0] += 'X'
            circuit[1].append(q + 1)
        elif h[q] == 1j:
            circuit[0] += 'Z'
            circuit[1].append(q + 1)
        elif h[q] == 1 + 1j:
            circuit[0] += 'Y'
            circuit[1].append(q + 1)
    return circuit

def report_error(j, m, h_ckt, h_new_ckt):
    """Report an error if the conjugation is incorrect."""
    if j <= m - k:
        print(f'\nSomething is wrong for logical Pauli X{j}!!')
    elif j > m:
        print(f'\nSomething is wrong for logical Pauli Z{j - m}!!')
    else:
        print(f'\nSomething is wrong for stabilizer {j - (m - k)}!!')

def get_cheapest_choice(choices, m):
    """Get the cheapest choice based on non-zero elements."""
    choices = choices[:, :m] + 1j * choices[:, m:]
    return choices[np.argmin(np.sum(choices != 0, axis=1))]

def append_single_qubit_gates(F_entry, x):
    """Append single-qubit gates to the circuit."""
    circuit = F_entry[1]
    if np.any(x.real == 1):
        circuit.append(('X', np.where(x.real == 1)[0] + 1))
    if np.any(x.imag == 1):
        circuit.append(('Z', np.where(x.imag == 1)[0] + 1))
    if np.any((x.real == 1) & (x.imag == 1)):
        circuit.append(('Y', np.where((x.real == 1) & (x.imag == 1))[0] + 1))
    F_entry[2] += 1  # Increase circuit depth




In [ ]:
import numpy as np

def find_logical_cliff(S, Xbar, Zbar, circuit, Snorm=None, no_of_solns=1):
    """
    Find all symplectic matrices for a logical Clifford operator.
    
    Parameters:
    - S: Stabilizer matrix
    - Xbar, Zbar: Logical operators
    - circuit: Circuit description as a list of tuples (Gate, Qubits)
    - Snorm: Normalized stabilizer (default is S)
    - no_of_solns: Number of solutions to find (default is 1)
    
    Returns:
    - F_all: List of symplectic matrices and their corresponding circuits
    """
    if Snorm is None:
        Snorm = S  # Assume physical operator must centralize S

    k, n = S.shape
    m = n // 2
    tot = 2 ** (k * (k + 1) // 2)
    F_all = []

    # Step 1: Initial symplectic transformation
    Fin = find_symplectic(m - k, circuit)
    H = np.vstack([Xbar, Snorm, Zbar])
    H[:2 * (m - k)] = mod2(Fin @ H[:2 * (m - k)])

    # Step 2: Complete symplectic basis for F_2^(2m)
    U = np.vstack([Xbar, S, Zbar])
    for i in range(k):
        h = np.zeros(2 * m - k + i)
        h[m - k + i] = 1
        U[2 * m - k + i] = gflineq(np.fft.fftshift(U, axes=1), h)

    if no_of_solns == 1:
        F_all.append([find_symp_mat(U[:2 * m - k], H), [], 0])
    else:
        F_all = [[sol, [], 0] for sol in qfind_all_symp_mat(U, H)]

    # Step 3: Process each solution
    for i, F_entry in enumerate(F_all):
        F, _, _ = F_entry
        circuit = find_circuit(F)
        F_all[i][1] = circuit  # Store circuit
        F_all[i][2] = len(circuit)  # Circuit depth

        # Verify signs under conjugation
        v = np.zeros(2 * m - k, dtype=int)
        for j in range(2 * m - k):
            h = U[j, :m] + 1j * U[j, m:]
            h_new = H[j, :m] + 1j * H[j, m:]
            
            h_ckt = convert_to_circuit(h, m)
            h_new_ckt = convert_to_circuit(h_new, m)

            in_sign = get_conjugate_sign(j, m, circuit, 'X' if j <= m - k else 'Z')
            h_new_ckt, out_sign = calc_conjugate(m, h_ckt, circuit)

            if h_new_ckt == h_ckt and out_sign == -in_sign:
                v[j] = 1
            else:
                report_error(j, m, h_ckt, h_new_ckt)

        # Handle non-trivial signs
        if np.any(v == 1):
            choices = np.fft.fftshift(gflineq_all(H, v), axes=1)
            x = get_cheapest_choice(choices, m)
            append_single_qubit_gates(F_all[i], x)

    return F_all

